In [13]:
import pandas as pd
import xml.etree.ElementTree as ET
import os
from fuzzywuzzy import fuzz

### from the dbs of
http://jujuvincebros.fr/telechargements2/file/10-arrm-another-recalbox-roms-manager

In [14]:
# mdb file was split into 3 files due to MS Acces export limitations
Game = pd.read_excel('Game.xlsx')
Game1 = pd.read_excel('Game1.xlsx')
Game3 = pd.read_excel('Game3.xlsx')

In [15]:
# combine all the dataframes into one
Game = pd.concat([Game, Game1, Game3], ignore_index=True)

In [16]:
Game.columns

Index(['Name', 'ReleaseYear', 'Overview', 'MaxPlayers', 'ReleaseType',
       'Cooperative', 'VideoURL', 'CommunityRating', 'Platform', 'ESRB',
       'CommunityRatingCount', 'Genres', 'Developer', 'Publisher', 'N°',
       'DatabaseID', 'WikipediaURL', 'AlternateName_Australia',
       'AlternateName_Brazil', 'AlternateName_China', 'AlternateName_Europe',
       'AlternateName_France', 'AlternateName_Germany', 'AlternateName_Japan',
       'AlternateName_Korea', 'AlternateName_NorthAmerica',
       'AlternateName_Spain', 'AlternateName_UnitedStates',
       'AlternateName_World'],
      dtype='object')

In [17]:
Game['Platform'].unique()

array(['Nintendo Game Boy Advance', 'Nintendo Game Boy Color',
       'Nintendo GameCube', 'Nintendo Pokemon Mini',
       'Nintendo Satellaview', 'Nintendo Switch', 'Nintendo Virtual Boy',
       'Nintendo Wii', 'Nintendo Wii U', 'Nokia N-Gage', 'Nuon',
       'OpenBOR', 'Oric Atmos', 'Othello Multivision', 'Ouya',
       'PC Engine SuperGrafx', 'Philips CD-i', 'Philips VG 5000',
       'Philips Videopac+', 'Pinball', 'RCA Studio II', 'SAM Coupé',
       'Sammy Atomiswave', 'ScummVM', 'Sega 32X', 'Sega CD',
       'Sega CD 32X', 'Sega Dreamcast', 'Sega Dreamcast VMU',
       'Sega Game Gear', 'Sega Genesis', 'Sega Hikaru',
       'Sega Master System', 'Sega Model 1', 'Sega Model 2',
       'Sega Model 3', 'Sega Naomi', 'Sega Naomi 2', 'Sega Pico',
       'Sega Saturn', 'Sega SC-3000', 'Sega SG-1000', 'Sega ST-V',
       'Sega System 16', 'Sega System 32', 'Sega Triforce',
       'Sharp MZ-2500', 'Sharp X1', 'Sharp X68000',
       'Sinclair ZX Spectrum', 'Sinclair ZX-81', 'SNK Neo Geo 

In [18]:
# missing values
Game.isnull().sum()

Name                               0
ReleaseYear                   149502
Overview                       19003
MaxPlayers                     54422
ReleaseType                    27485
Cooperative                        0
VideoURL                      152920
CommunityRating                67180
Platform                           0
ESRB                           61149
CommunityRatingCount               0
Genres                         17336
Developer                      24918
Publisher                      23723
N°                            101100
DatabaseID                    101100
WikipediaURL                  197030
AlternateName_Australia       232956
AlternateName_Brazil          232950
AlternateName_China           233100
AlternateName_Europe          233100
AlternateName_France          232078
AlternateName_Germany         231082
AlternateName_Japan           224292
AlternateName_Korea           232737
AlternateName_NorthAmerica    233100
AlternateName_Spain           231180
A

In [19]:
# Read the platform mappings from a text file and create a dictionary
platform_mappings = {}
with open('systems.txt', 'r') as mapping_file:
    for line in mapping_file:
        abbreviated, full_name = line.strip().split(': ')
        platform_mappings[abbreviated] = full_name

In [21]:
# CommunityRating max and min
print(Game['CommunityRating'].max())
print(Game['CommunityRating'].min())

5.0
0.5


In [22]:
# import video game dataset all_games.csv with metacritic scores
mc_df = pd.read_csv('all_games.csv')

In [26]:
mc_df.head()

,name,platform,release_date,summary,meta_score,user_review,meta_score_ten
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",99,9.1,9.9
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,98,7.4,9.8
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7,9.8
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",98,8.4,9.8
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9,9.8


In [24]:
mc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18800 entries, 0 to 18799
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          18800 non-null  object
 1   platform      18800 non-null  object
 2   release_date  18800 non-null  object
 3   summary       18686 non-null  object
 4   meta_score    18800 non-null  int64 
 5   user_review   18800 non-null  object
dtypes: int64(1), object(5)
memory usage: 881.4+ KB


In [32]:
# remove rows with user_review as tbd
mc_df = mc_df[mc_df['user_review'] != 'tbd']

In [36]:
mc_df.value_counts('user_review')

user_review
7.7    727
7.8    701
7.5    682
7.6    673
7.3    632
      ... 
0.6      2
0.9      1
0.7      1
9.4      1
0.2      1
Name: count, Length: 93, dtype: int64

In [35]:
# user review to int64
mc_df['user_review'] = mc_df['user_review'].astype('int64')

ValueError: invalid literal for int() with base 10: '9.1'

In [25]:
mc_df['meta_score_ten'] = mc_df['meta_score'] / 10

In [28]:
# create a new column with the average of the user and critic scores
mc_df['avg_score'] = (int(mc_df['user_review'] + mc_df['meta_score_ten']) / 2

TypeError: can only concatenate str (not "float") to str

In [9]:
# scale CommunityRating to between 0 and 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
Game['CommunityRating'] = scaler.fit_transform(Game['CommunityRating'].values.reshape(-1,1))


In [10]:
Game['CommunityRating'].describe()

count    165920.000000
mean          0.670623
std           0.208643
min           0.000000
25%           0.555556
50%           0.688889
75%           0.811966
max           1.000000
Name: CommunityRating, dtype: float64

In [11]:
column_to_element = {
    'Overview': 'desc',
    'Developer': 'developer',
    'Genres': 'genre',
    'Name': 'name',
    'Publisher': 'publisher',
    'CommunityRating': 'rating',
    'ReleaseYear': 'releasedate',
    'MaxPlayers': 'players'
}

In [12]:
def update_or_create_element(parent_elem, element_name, text):
    elem = parent_elem.find(element_name)
    if elem is not None:
        elem.text = text
    else:
        new_elem = ET.Element(element_name)
        new_elem.text = text
        parent_elem.append(new_elem)

In [34]:
# Iterate through each gamelist.xml file in lists/[platform folder]
new_list_name = 'gamelist_updated.xml'
base_path = 'lists'

for platform_path in os.listdir(base_path):
    if os.path.isdir(os.path.join(base_path, platform_path)):
        xml_path = os.path.join(base_path, platform_path, 'gamelist.xml')
        updated_xml_path = os.path.join(platform_path, new_list_name)
        if os.path.exists(xml_path) # and not os.path.exists(updated_xml_path):
            
            # identify the platform using the directory name
            platform_abbreviated = os.path.basename(platform_path)
            print(f"Creating {platform_abbreviated}/{new_list_name}")

            full_platform_name = platform_mappings.get(platform_abbreviated, platform_abbreviated)

            # subset dataframe to only include games for the current platform
            platform_subset = Game[Game['Platform'] == full_platform_name]

            # parse the xml file
            tree = ET.parse(xml_path)
            xml_root = tree.getroot()

            for game_elem in xml_root.findall('game'):
                title_xml = game_elem.find('name').text
                best_match = None
                best_match_score = 0

                # find best fuzzy match
                for _, row in platform_subset.iterrows():
                    title_df = row['Name']
                    score = fuzz.ratio(title_xml, title_df)
                    if score > best_match_score:
                        best_match = row
                        best_match_score = score
                
                # update the xml file with the best match
                if best_match is not None:
                    for column, element in column_to_element.items():
                        column_value = best_match[column]
                        if not pd.isna(column_value):
                            update_or_create_element(game_elem, element, str(column_value))
        
            # write the updated xml file
            updated_xml_path = os.path.join(base_path, platform_path, new_list_name)
            tree.write(updated_xml_path, encoding='utf-8')


Creating genesis/gamelist_updated.xml
Creating mame/gamelist_updated.xml
Creating fbneo/gamelist_updated.xml
Creating psx/gamelist_updated.xml
Creating dreamcast/gamelist_updated.xml
Creating gc/gamelist_updated.xml
Creating sfc/gamelist_updated.xml
Creating pc/gamelist_updated.xml
Creating tg16/gamelist_updated.xml
Creating saturn/gamelist_updated.xml
Creating nes/gamelist_updated.xml
Creating snes/gamelist_updated.xml
Creating scummvm/gamelist_updated.xml
Creating gba/gamelist_updated.xml
Creating msx/gamelist_updated.xml
Creating segacd/gamelist_updated.xml
Creating c64/gamelist_updated.xml
Creating zxspectrum/gamelist_updated.xml
Creating psp/gamelist_updated.xml
Creating n64/gamelist_updated.xml
Creating pcengine/gamelist_updated.xml
Creating fba/gamelist_updated.xml
Creating mastersystem/gamelist_updated.xml
Creating atari2600/gamelist_updated.xml
Creating ngpc/gamelist_updated.xml
Creating doom/gamelist_updated.xml
Creating gamegear/gamelist_updated.xml
Creating ngp/gamelist_upd

In [35]:
import pickle
with open('game_df.pkl' , 'wb') as f:
    pickle.dump(Game, f)

In [8]:
# import gamelist.xml
with open('gamelist.xml', 'r') as f:
    soup = BeautifulSoup(f, 'xml')

In [9]:
xml_titles = soup.find_all('name')


In [10]:
# create a list of lowercase titles from the <game><name> tags
xml_titles_list = []
for title in xml_titles:
    xml_titles_list.append(title.text.lower())

In [11]:
xml_titles_list[:5]

['118-in-1 [p1][!]',
 '10-yard fight',
 '10-yard fight',
 '1942',
 '1943 : the battle of midway']

In [13]:
platform_df.head()

NameError: name 'platform_df' is not defined

In [22]:
platform_name = 'Nintendo Entertainment System'

# create a list of titles from the Game dataframe
platform_df = Game[Game['Platform'] == platform_name]
platform_df['Name'] = platform_df['Name'].str.lower()


/tmp/ipykernel_7881/843215976.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  platform_df['Name'] = platform_df['Name'].str.lower()


In [23]:
platform_df.head()

,Name,ReleaseYear,Overview,MaxPlayers,ReleaseType,Cooperative,VideoURL,CommunityRating,Platform,ESRB,...,AlternateName_China,AlternateName_Europe,AlternateName_France,AlternateName_Germany,AlternateName_Japan,AlternateName_Korea,AlternateName_NorthAmerica,AlternateName_Spain,AlternateName_UnitedStates,AlternateName_World
95167,0-to-x,2015.0,"Based on a hit internet phenomenon, 0-to-X is ...",1.0,Homebrew,False,https://www.youtube.com/watch?v=uLLVUH65Bqk,2.678571,Nintendo Entertainment System,Not Rated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95168,1007 bolts,NaN,So you've pissed off the Gods... Now what?\n\n...,1.0,Homebrew,False,NaN,3.375000,Nintendo Entertainment System,Not Rated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95169,100-in-1 contra 16 function,1991.0,100-in-1 Contra 16 Function is a Famicom multi...,2.0,Unlicensed,True,NaN,3.100000,Nintendo Entertainment System,Not Rated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95170,10-yard fight,NaN,The main idea of this game is that players tak...,2.0,Released,False,https://www.youtube.com/watch?v=5LUyTv9j1TM,3.082090,Nintendo Entertainment System,Not Rated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95171,110-in-1,1990.0,NaN,NaN,Unlicensed,False,NaN,3.555556,Nintendo Entertainment System,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
platform_titles = platform_df['Name'].tolist()

In [28]:
match_count = 0
for title_xml in xml_titles_list:
    if any(fuzz.ratio(title_xml, title_platform) > 90 for title_platform in platform_titles):
        match_count += 1

print(match_count)

1657


In [29]:
len(xml_titles_list)

1916

In [6]:
def get_fuzzy(title_a, title_b):
    return fuzz.ratio(title_a, title_b)